# 安装google云盘必要的包

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!nvidia-smi

Fri Mar 20 10:53:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# 连接google云盘

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls drive/kaikeba/Abstract/model/pgn

layers.py  pgn_model.py  __pycache__


In [0]:
!cat drive/kaikeba/Abstract/model/pgn/pgn_model.py

# 载入代码

In [0]:
! pip install rouge

In [0]:
! pip install tensorflow-gpu==2.0.0

In [0]:
import tensorflow as tf
tf.__version__

In [0]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('drive/kaikeba/Abstract/')

# 导入训练代码

In [0]:
from utils.textprocessing import load_embedding_matrix
from utils.data_loader import load_data
from model.pgn.pgn_model import AttentionModel
import pandas as pd

# 预处理数据

In [0]:
# build_dataset(train_data_path, test_data_path)

# 参数设置

In [0]:
!nvidia-smi

In [0]:
embedding_matrix, word_index_dict = load_embedding_matrix()
# train_x, train_y, test_x = load_data()

In [0]:
train_x = pd.read_csv('drive/kaikeba/Abstract/data/train_X_seg_data.csv', encoding='utf-8', squeeze=True, header=None)
with open('drive/kaikeba/Abstract/data/train_Y_seg_data.csv', 'r') as file:
  content = file.read()
  train_y = content.splitlines()
# train_y = pd.read_csv('drive/kaikeba/Abstract/data/train_Y_seg_data.csv', encoding='utf-8', squeeze=True, header=None)

In [0]:
temp = []
for y in train_y:
  y_list = y.split()
  temp_str = '<start> ' + ' '.join(y_list[:min(32, len(y_list))]) +' <end>'
  temp.append(temp_str)
train_y = pd.Series(temp) 

# 训练模型

In [0]:
# GPU 700s TPU x s
x = train_x
y = train_y
model = AttentionModel(embedding_matrix=embedding_matrix, word_index_dict=word_index_dict, max_length_input=200, max_length_output=34, batch_size=32, epochs=4, encoder_units=256, attention_units=256, decoder_units=256, learning_rate=0.15)
checkpoint = tf.train.Checkpoint(PNG=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, 'drive/kaikeba/Abstract/data/checkpoints/test2', max_to_keep=5)
# checkpoint_manager.save()
checkpoint.restore(checkpoint_manager.latest_checkpoint)
model.fit(x, y)
checkpoint = tf.train.Checkpoint(PNG=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, 'drive/kaikeba/Abstract/data/checkpoints/test2', max_to_keep=5)
checkpoint_manager.save()

# 生成结果

In [0]:
def submit_proc(sentence):
    sentence = sentence.replace('<start>', '').replace('<end>', '').replace('<pad>', '')
    sentence=sentence.lstrip(' ，！。')
    sentence=sentence.replace(' ','')
    if sentence=='':
        sentence='随时联系'
    return sentence

In [0]:
model = AttentionModel(embedding_matrix=embedding_matrix, word_index_dict=word_index_dict, max_length_input=200, max_length_output=34, batch_size=32, epochs=10, encoder_units=256, attention_units=256, decoder_units=256, learning_rate=0.15)
checkpoint = tf.train.Checkpoint(PNG=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, 'drive/kaikeba/Abstract/data/checkpoints/test2', max_to_keep=5)
# checkpoint_manager.save()
checkpoint.restore(checkpoint_manager.latest_checkpoint)

In [0]:
test_x = pd.read_csv('drive/kaikeba/Abstract/data/test_X_seg_data.csv', encoding='utf-8', squeeze=True, header=None)

In [0]:
test_y = []
for i in range(len(test_x)):
  print(i)
  test_y.append(model.beam_predict(test_x[i: i + 1])[0])
test_y = [submit_proc(item) for item in test_y]

In [0]:
idx_list = []
for i in range(len(test_y)):
  idx_list.append('Q{}'.format(i + 1))
result_df = pd.DataFrame()
result_df['QID'] = pd.Series(idx_list)
result_df['Prediction'] = pd.Series(test_y)
result_df.to_csv('drive/kaikeba/Abstract/data/result_scheduled_sampling.csv', encoding='utf-8', index=False)